## 누락 데이터 식별과 정제
- 평균으로 채우기, 이웃값의 정방향 채우기 등 누락 데이터를 처리하는 모든 전략 탐구

In [1]:
import pandas as pd

nls97=pd.read_csv('C:/data-cleansing-main/Chapter06/data/nls97c.csv')
nls97.set_index('personid',inplace=True)

In [25]:
schoolrecordlist = ['satverbal','satmath','gpaoverall','gpaenglish','gpamath','gpascience','highestdegree','highestgradecompleted']

demolist = ['maritalstatus','childathome','childnotathome','wageincome','weeklyhrscomputer','weeklyhrstv','nightlyhrssleep']
schoolrecord = nls97[schoolrecordlist]
demo=nls97[demolist]

In [27]:
schoolrecord.shape
demo.shape

(8984, 7)

In [28]:
schoolrecord.isna().sum(axis=0)             # 각 열에 있는 누락값의 개수

satverbal                7578
satmath                  7577
gpaoverall               2980
gpaenglish               3186
gpamath                  3218
gpascience               3300
highestdegree              31
highestgradecompleted    2321
dtype: int64

In [29]:
misscnt=schoolrecord.isna().sum(axis=1)     # 각 행에 있는 누락값의 개수
misscnt.value_counts().sort_index()

0    1087
1     312
2    3210
3    1102
4     176
5     101
6    2039
7     946
8      11
dtype: int64

In [16]:
schoolrecord.loc[misscnt>=7].head(4).T          # 이렇게가 되는구나..?

personid,101705,102061,102648,104627
satverbal,NaN,NaN,NaN,NaN
satmath,NaN,NaN,NaN,NaN
gpaoverall,NaN,NaN,NaN,NaN
gpaenglish,NaN,NaN,NaN,NaN
gpamath,NaN,NaN,NaN,NaN
gpascience,NaN,NaN,NaN,NaN
highestdegree,1. GED,0. None,1. GED,0. None
highestgradecompleted,NaN,NaN,NaN,NaN


In [30]:
schoolrecord=schoolrecord.dropna(thresh=2)      # 비누락값이 2개 이하인 행이 삭제됨

In [31]:
schoolrecord.isna().sum(axis=1).value_counts().sort_index()

0    1087
1     312
2    3210
3    1102
4     176
5     101
6    2039
dtype: int64

### 평균값으로 채우기

In [32]:
gpamean=int(schoolrecord.gpaoverall.mean())

schoolrecord.gpaoverall.fillna(gpamean,inplace=True)
schoolrecord.gpaoverall.isna().sum()

0

### 정방향 채우기: fillna의 ffill옵션을 사용해 누락값 이전에 나온 가장 가까운 비누락값으로 대체
- 가장 가깝다는게 뭐지?
- 데이터 값들이 인접한 행과 공통점이 있다는 근거가 있다면, 정방향 채우기를 추천

In [34]:
demo.wageincome.head().T

personid
100061     12500.0
100139    120000.0
100284     58000.0
100292         NaN
100583     30000.0
Name: wageincome, dtype: float64

In [36]:
nls97.wageincome.fillna(method='ffill',inplace=True)
demo=nls97[demolist]
demo.wageincome.head().T

personid
100061     12500.0
100139    120000.0
100284     58000.0
100292     58000.0
100583     30000.0
Name: wageincome, dtype: float64

### 그룹별 평균으로 채우기
- 관심 변수가 그룹 소속에 따라 상당한 차이를 보인다면 그룹 평균을추천

In [37]:
nls97[['highestdegree','weeksworked17']].head()

,highestdegree,weeksworked17
personid,,
100061,2. High School,48.0
100139,2. High School,52.0
100284,0. None,0.0
100292,4. Bachelors,NaN
100583,2. High School,52.0


In [41]:
workbydegree=nls97.groupby(['highestdegree'])['weeksworked17'].mean().reset_index().rename(columns={'weeksworked17':'meanweeksworked17'})
nls97=nls97.reset_index().merge(workbydegree,left_on=['highestdegree'],right_on=['highestdegree'],how='left').set_index('personid')
nls97.weeksworked17.fillna(nls97.meanweeksworked17,inplace=True)
nls97[['highestdegree','weeksworked17','meanweeksworked17']].head()

,highestdegree,weeksworked17,meanweeksworked17
personid,,,
100061,2. High School,48.000000,38.150469
100139,2. High School,52.000000,38.150469
100284,0. None,0.000000,28.719608
100292,4. Bachelors,43.565574,43.565574
100583,2. High School,52.000000,38.150469
